In [10]:
import numpy as np

In [61]:
class Board(object):
    def __init__(self, player_sym='x'):
        self.board = None
        self.reset_board()
        self.sym_o = {
            'mark': 'O',
            'value': 1
        }
        self.sym_x = {
            'mark': 'X',
            'value': 2
        }
        self.sym_empty = {
            'mark': ' ',
            'value': 0
        }
        self.player_sym, self.bot_sym = (self.sym_x, self.sym_o) \
                                        if player_sym.lower() == 'x' \
                                        else (self.sym_o, self.sym_x)
        self.winner = None

    def reset_board(self):
        self.board = np.zeros((3, 3))
        self.winner = None
        
    def draw_char_for_item(self, item):
        if item == self.sym_x.get('value'):
            return self.sym_x.get('mark')
        elif item == self.sym_o.get('value'):
            return self.sym_o.get('mark')
        else:
            return self.sym_empty.get('mark')

    def draw_board(self):
        elements_in_board = self.board.size
        items = [
            self.draw_char_for_item(self.board.item(item_idx)) 
            for item_idx in range(elements_in_board)
        ]
        board = """
             {} | {} | {}
            -----------
             {} | {} | {}
            -----------
             {} | {} | {}
        """.format(*items)
        print(board)
        
    def have_same_val(self, axis, item, item_x, item_y):
        max_limit, _ = self.board.shape
        result = True
        row_idx = col_idx = 0
        main_idx, fixed_idx, ignore_idx = (col_idx, item_x, item_y) \
                                            if axis == 0 \
                                            else (row_idx, item_y, item_x)
        while main_idx < max_limit:
            if main_idx != ignore_idx:
                board_item = self.board[fixed_idx][main_idx] \
                    if axis == 0 \
                    else self.board[main_idx][fixed_idx]
                
                if board_item != item or board_item == 0:
                    result = False
                    break
            main_idx += 1
        return result
    
    def left_diagonal_has_same_values(self, item, item_x, item_y):
        i = j = 0
        result = True
        max_limit, _ = self.board.shape
        while i < max_limit:
            if i != item_x:
                if self.board[i][j] != item or self.board[i][j] == 0:
                    result = False
                    break
            i += 1
            j += 1
        return result

    def right_diagonal_has_same_values(self, item, item_x, item_y):
        result = True
        max_limit, _ = self.board.shape
        i = 0
        j = max_limit - 1
        while i < max_limit:
            if i != item_x:
                if self.board[i][j] != item or self.board[i][j] == 0:
                    result = False
                    break
            i += 1
            j -= 1
        return result

    def cols_have_same_values(self, item, item_x, item_y):
        axis = 1
        return self.have_same_val(axis, item, item_x, item_y)

    def rows_have_same_values(self, item, item_x, item_y):
        axis = 0
        return self.have_same_val(axis, item, item_x, item_y)
    
    def element_diagonal_has_same_value(self, item, item_x, item_y):
        max_limit, _ = self.board.shape
        if item_x == item_y:
            return self.left_diagonal_has_same_values(item, item_x, item_y)
        elif item_x + item_y == max_limit - 1:
            return self.right_diagonal_has_same_values(item, item_x, item_y)
        else:
            return False
    
    def is_game_over(self, player, item, item_x, item_y):
        return self.cols_have_same_values(item, item_x, item_y) or \
                    self.rows_have_same_values(item, item_x, item_y) or \
                    self.element_diagonal_has_same_value(item, item_x, item_y)

    def is_winning_move(self, player, item, item_x, item_y):
        if self.is_game_over(player, item, item_x, item_y):
            self.winner = player
            return True
        return False
    
    def player_move(self, input_symbol, item_x, item_y):
        symbol = None
        if input_symbol == self.sym_o.get('mark'):
            symbol = self.sym_o
        elif input_symbol == self.sym_x.get('mark'):
            symbol = self.sym_x
        else:
            return
        self.board[item_x][item_y] = symbol.get('value')
        self.draw_board()
        if self.is_winning_move(symbol.get('mark'), symbol.get('value'), item_x, item_y):
            print('Winner is: {}'.format(self.winner))
        
        
    def play(self, item_x, item_y):
        max_limit, _ = self.board.shape
        if item_x > max_limit - 1 or item_y > max_limit:
            return
        self.player_move(self.player_sym.get('mark'), item_x, item_y)
        
    def bot_play(self, item_x, item_y):
        max_limit, _ = self.board.shape
        if item_x > max_limit - 1 or item_y > max_limit:
            return
        self.player_move(self.bot_sym.get('mark'), item_x, item_y)

In [89]:
b = Board()


               |   | X
            -----------
               |   |  
            -----------
               |   |  
        

             X |   | X
            -----------
               |   |  
            -----------
               |   |  
        

             X |   | X
            -----------
               |   | X
            -----------
               |   |  
        

             X |   | X
            -----------
               |   | X
            -----------
               | X |  
        

             X |   | X
            -----------
               |   | X
            -----------
               | X | X
        
Winner is: X



               |   | X
            -----------
               |   | X
            -----------
               |   |  
        



               |   | X
            -----------
               |   | X
            -----------
               |   | X
        
Winner is: X
